In [1]:
# 2017013207 电商1701 丁泽锋
# 2020-05-02
import os
import sys
spark_name = os.environ.get('SPARK_HOME',None)
if not spark_name:
    raise ValueErrorError('spark环境没有配置好')
sys.path.insert(0,os.path.join(spark_name,'python'))
sys.path.insert(0,os.path.join(spark_name,'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_name,'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Python version 3.5.2 (default, Jul  5 2016 11:41:13)
SparkSession available as 'spark'.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('learn_regression').master('local[1]').getOrCreate()
# 数据导入
df_train = spark.read.csv('D:/code/Spark/data/train.csv', header=True, inferSchema=True, encoding='utf-8')
df_test = spark.read.csv('D:/code/Spark/data/test.csv', header=True, inferSchema=True, encoding='utf-8')
# 表合并
from pyspark.sql.functions import lit
df_test = df_test.withColumn('medv', lit(22.77))
df0 = df_train.union(df_test).sort('ID')
df0.show(3)


+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+-----+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat| medv|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+-----+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98| 24.0|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14| 21.6|
|  3|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|22.77|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+-----+
only showing top 3 rows



In [4]:
from pyspark.ml.feature import VectorAssembler
def feature_converter(df):
    vecAss = VectorAssembler(inputCols=df0.columns[1:-1], outputCol='features')
    df_va = vecAss.transform(df)
    return df_va

train_data, test_data = feature_converter(df0).select(['features', 'medv']).randomSplit([7.0, 3.0], 101)


In [6]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(maxDepth=5, varianceCol="variance", labelCol='medv')
dt_model = dt.fit(train_data)

dt_model.featureImportances


SparseVector(13, {0: 0.0503, 2: 0.011, 4: 0.0622, 5: 0.1441, 6: 0.1852, 7: 0.0262, 8: 0.0022, 9: 0.0886, 10: 0.0142, 12: 0.4159})

In [7]:
result = dt_model.transform(test_data)
result.show(3)

+--------------------+-----+------------------+------------------+
|            features| medv|        prediction|          variance|
+--------------------+-----+------------------+------------------+
|[0.03237,0.0,2.18...| 33.4| 34.12833333333334|29.509013888888756|
|[0.08829,12.5,7.8...| 22.9|21.195135135135136| 4.446162819576342|
|[0.14455,12.5,7.8...|22.77|22.425999999999995|0.5578440000003866|
+--------------------+-----+------------------+------------------+
only showing top 3 rows



In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
dt_evaluator = RegressionEvaluator(labelCol='medv', metricName="rmse", predictionCol='prediction')
rmse = dt_evaluator.evaluate(result)
print('测试数据的均方根误差（rmse）:{}'.format(rmse))


测试数据的均方根误差（rmse）:6.555920141221407
